In [ ]:
Intro

Creating Deployments
- nemotron nano 9B v2
- nemotron nano 12B v2

Show how to create deployments

In [ ]:
Performance Benchmarking
- nemotron nano 9B v2
- nemotron nano 12B v2
- qwen3 8B
- qwen3 14B
- llama 3.1 8B

Create deployments of the other 3 and run benchmark; plot results

In [ ]:
Accuracy Benchmarking
- nemotron nano 9B v2
- nemotron nano 12B v2
- qwen3 8B
- qwen3 14B
- llama 3.1 8B

Using deployments, run an accuracy benchmark (GPQA diamond) for each model

In [ ]:
Qualitative Examples
- nemotron nano 9B v2
- nemotron nano 12B v2
- qwen3 8B
- qwen3 14B
- llama 3.1 8B

Using examples, show some qualitative inputs and outputs comparing model behavior


In [ ]:
Conclusions